# Modellerin Değerlendirilmesi ve Seçimi

* Modellerin başarısı, parametrelere bağlıdır
* Makine öğrenmesi parametreleri optimize etmez
* Model seçiminden önce dikkat edilmesi gereken ilk nokta, model değerlendirmesidir (evaluation)
* Şimdiye kadar tst kümesindeki başarı ölçüldü
* k-fold Cross Validation (k-katlamalı Çapraz Doğrulama)
* Izgara Araması (Grid Search)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
veriler = pd.read_csv("../Docs/Social_Network_Ads.csv")
X = veriler.iloc[:, [2, 3]].values
y = veriler.iloc[:, 4].values

# Eğitim ve test kümelerinin bölünmesi
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

# Ölçekleme
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# SVM
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

# Tahminler
y_pred = classifier.predict(X_test)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

print(cm)

[[64  4]
 [ 3 29]]


In [3]:
?classifier

Type:        SVC
String form: SVC(random_state=0)
File:        c:\users\tolga\appdata\local\programs\python\python310\lib\site-packages\sklearn\svm\_classes.py
Docstring:  
C-Support Vector Classification.

The implementation is based on libsvm. The fit time scales at least
quadratically with the number of samples and may be impractical
beyond tens of thousands of samples. For large datasets
consider using :class:`~sklearn.svm.LinearSVC` or
:class:`~sklearn.linear_model.SGDClassifier` instead, possibly after a
:class:`~sklearn.kernel_approximation.Nystroem` transformer.

The multiclass support is handled according to a one-vs-one scheme.

For details on the precise mathematical formulation of the provided
kernel functions and how `gamma`, `coef0` and `degree` affect each
other, see the corresponding section in the narrative documentation:
:ref:`svm_kernels`.

Read more in the :ref:`User Guide <svm_classification>`.

Parameters
----------
C : float, default=1.0
    Regularization parame

In [12]:
# K-katlamalı çapraz doğrulama 
''' 
1. estimator : classifier (kullanılacak algoritma)
2. X
3. Y
4. cv : kaç katlamalı
'''
basari = cross_val_score(estimator = classifier, X=X_train, y=y_train , cv = 4)
print(f"Ortalama: {basari.mean()}")
print(f"Standart Sapma: {basari.std()}")

Ortalama: 0.8999999999999999
Standart Sapma: 0.03590109871423001


# Hangi Model Seçilmeli

* **Adım 1:** Model Tipinin Belirlenmesi
    * Bağımlı Değişken Var mı? (Classification-Regression / Clustering)
        * Bağımlı Değişken Kategorik mi yoksa Sürekli bir sayı mı? (Classification / regression)
    * Doğrusal/Doğrusal olmayan

In [15]:
# Parametremetre optimizasyonu ve algoritma seçimi
p = [{'C':[1,2,3,4,5],'kernel':['linear']},
     {'C':[1,2,3,4,5] ,'kernel':['rbf'],
      'gamma':[1,0.5,0.1,0.01,0.001]} ]

'''
GSCV parametreleri
estimator : sınıflandırma algoritması (neyi optimize edileceğini belirleme)
param_grid : parametreler/ denenecekler
scoring: neye göre skorlanacak : örn : accuracy
cv : kaç katlamalı olacağı
n_jobs : aynı anda çalışacak iş
'''
gs = GridSearchCV(estimator= classifier, #SVM algoritması
                  param_grid = p,
                  scoring =  'accuracy',
                  cv = 10,
                  n_jobs = -1)

grid_search = gs.fit(X_train,y_train)
eniyisonuc = grid_search.best_score_
eniyiparametreler = grid_search.best_params_

print(f"En iyi sonuç: {eniyisonuc}")
print(f"En iyi parametreler: {eniyiparametreler}")

En iyi sonuç: 0.9099999999999999
En iyi parametreler: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
